In [19]:
!pip install tqdm

In [20]:
import bs4
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
from tqdm import tqdm

In [21]:
def GenerateCourse(table, dept):
  #going throught the entire thing to get all the sections
  lines = table.get_text().splitlines()

  #getting the course information

  try:
    while dept not in lines[0] :
          lines.pop(0)
  except IndexError:
        pass

  course = {
      "Name": lines.pop(0),
      "Sections": []
  }
  section = {}

  #removing all the fluff
  try:
    while lines[0] != "تفاصيل":
      lines.pop(0)
  except IndexError:
        pass

  lines.pop(0)
  lines.pop(0)
  lines.pop(0)
  #getting the sections
  while len(lines) > 0:

      section["Code"] = lines.pop(0)
      section["Section"] = lines.pop(0)
      section["Class_type"] = lines.pop(0)
      section["Time"] = lines.pop(0)
      section["Days"] = lines.pop(0)
      section["Building"] = lines.pop(0)
      section["Room"] = lines.pop(0)
      section["Teacher"] = lines.pop(0)
      if len(lines) == 0:
            break
      lines.pop(0)

      try:
        while len(lines[0]) <5 :
          lines.pop(0)
      except IndexError:
        pass

      #checking if there is a lab
      if len(lines[0]) > 5:
          section["Lab_Time"] = lines.pop(0)
          section["Lab_Days"] = lines.pop(0)
          section["Lab_Building"] = lines.pop(0)
          section["Lab_Room"] = lines.pop(0)
          section["Lab_Teacher"] = lines.pop(0)
          if len(lines) == 0:
            break
          lines.pop(0)

      try:
        while len(lines[0]) <5 :
          lines.pop(0)
      except IndexError:
        pass

      course["Sections"].append(section)
      section = {}
      if len(lines) == 0:
        break
  return course




In [22]:
courses=[]
deptList = ["CPCS", "CPIT", "CPIS"]
courseCodes = range(200, 500)
for dept in deptList:
  for course in tqdm(courseCodes, desc=f'Processing {dept}', unit='course'):
    if course == 323:
      continue
    link = "https://odusplus-ss.kau.edu.sa/PROD/xwckctlg.p_display_courses2?term_in=202502&one_subj="+dept+"&sel_subj=&sel_crse_strt="+str(course)+"&sel_crse_end="+str(course)+"&sel_levl=&sel_schd=&sel_divs=&sel_coll=&sel_dept=&sel_attr="
    req = requests.get(link)
    beautifull = BeautifulSoup(req.text, "html.parser")
    if "تعذر العثور على مقررات تتطابق" in beautifull.get_text() or  "لم يتم العثور على فصول" in beautifull.get_text():
      continue
    course = GenerateCourse(beautifull, dept)
    courses.append(course)
  print(dept , "Completed")





Processing CPCS: 100%|██████████| 300/300 [02:11<00:00,  2.28course/s]


CPCS Completed


Processing CPIT: 100%|██████████| 300/300 [02:09<00:00,  2.31course/s]


CPIT Completed


Processing CPIS: 100%|██████████| 300/300 [02:10<00:00,  2.30course/s]

CPIS Completed


In [23]:
main_df = pd.DataFrame()
for course in courses:
  df = pd.DataFrame(course["Sections"])
  df.insert(0, "Course_Name", course["Name"])
  main_df = pd.concat([main_df, df])
main_df

,Course_Name,Code,Section,Class_type,Time,Days,Building,Room,Teacher,Lab_Time,Lab_Days,Lab_Building,Lab_Room,Lab_Teacher
0,CPCS 202 - برمجه 1,70959,AS,نظري/عملي,8:00 AM - 8:50 AM,TRU,007,087C,TBA,12:00 PM - 1:20 PM,R,07N,L04B,TBA
1,CPCS 202 - برمجه 1,64178,BAR,نظري/عملي,8:00 AM - 9:15 AM,MW,061,124F,TBA,9:30 AM - 10:50 AM,R,061,123G,TBA
2,CPCS 202 - برمجه 1,56801,CS1,نظري/عملي,1:00 PM - 1:50 PM,TRU,031 كلية الحاسبات وت.المعلومات,06,TBA,5:30 PM - 6:50 PM,R,031 كلية الحاسبات وت.المعلومات,06,TBA
3,CPCS 202 - برمجه 1,56275,CS2,نظري/عملي,2:30 PM - 3:50 PM,MW,471 ( فصول دراسية ميدان جديد),210A,TBA,5:30 PM - 6:50 PM,M,031 كلية الحاسبات وت.المعلومات,10,TBA
4,CPCS 202 - برمجه 1,56276,CS3,نظري/عملي,4:00 PM - 5:20 PM,MW,031 كلية الحاسبات وت.المعلومات,10,TBA,11:00 AM - 12:20 PM,U,031 كلية الحاسبات وت.المعلومات,04,TBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,CPIS 499 - مشروع تخرج 2,56605,S56,نظري,7:00 AM - 7:50 AM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN
76,CPIS 499 - مشروع تخرج 2,56606,S57,نظري,8:00 AM - 8:50 AM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN
77,CPIS 499 - مشروع تخرج 2,56607,S58,نظري,9:00 PM - 9:50 PM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN
78,CPIS 499 - مشروع تخرج 2,56608,S59,نظري,10:00 PM - 10:50 PM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN


In [24]:
main_df.to_csv("courses.csv", sep=",", index=False)

In [28]:
main_df[['Start Time', 'End Time']] = main_df['Time'].str.split(' - ', expand=True)

In [30]:
main_df[['Lab Start', 'Lab End']] = main_df['Lab_Time'].str.split(' - ', expand=True)

In [31]:
main_df

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Course_Name,Code,Section,Class_type,Time,Days,Building,Room,Teacher,Lab_Time,Lab_Days,Lab_Building,Lab_Room,Lab_Teacher,Start Time,End Time,Lab Start,Lab End
0,CPCS 202 - برمجه 1,70959,AS,نظري/عملي,8:00 AM - 8:50 AM,TRU,007,087C,TBA,12:00 PM - 1:20 PM,R,07N,L04B,TBA,8:00 AM,8:50 AM,12:00 PM,1:20 PM
1,CPCS 202 - برمجه 1,64178,BAR,نظري/عملي,8:00 AM - 9:15 AM,MW,061,124F,TBA,9:30 AM - 10:50 AM,R,061,123G,TBA,8:00 AM,9:15 AM,9:30 AM,10:50 AM
2,CPCS 202 - برمجه 1,56801,CS1,نظري/عملي,1:00 PM - 1:50 PM,TRU,031 كلية الحاسبات وت.المعلومات,06,TBA,5:30 PM - 6:50 PM,R,031 كلية الحاسبات وت.المعلومات,06,TBA,1:00 PM,1:50 PM,5:30 PM,6:50 PM
3,CPCS 202 - برمجه 1,56275,CS2,نظري/عملي,2:30 PM - 3:50 PM,MW,471 ( فصول دراسية ميدان جديد),210A,TBA,5:30 PM - 6:50 PM,M,031 كلية الحاسبات وت.المعلومات,10,TBA,2:30 PM,3:50 PM,5:30 PM,6:50 PM
4,CPCS 202 - برمجه 1,56276,CS3,نظري/عملي,4:00 PM - 5:20 PM,MW,031 كلية الحاسبات وت.المعلومات,10,TBA,11:00 AM - 12:20 PM,U,031 كلية الحاسبات وت.المعلومات,04,TBA,4:00 PM,5:20 PM,11:00 AM,12:20 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,CPIS 499 - مشروع تخرج 2,56605,S56,نظري,7:00 AM - 7:50 AM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN,7:00 AM,7:50 AM,NaN,NaN
76,CPIS 499 - مشروع تخرج 2,56606,S57,نظري,8:00 AM - 8:50 AM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN,8:00 AM,8:50 AM,NaN,NaN
77,CPIS 499 - مشروع تخرج 2,56607,S58,نظري,9:00 PM - 9:50 PM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN,9:00 PM,9:50 PM,NaN,NaN
78,CPIS 499 - مشروع تخرج 2,56608,S59,نظري,10:00 PM - 10:50 PM,TRU,TBA,TBA,TBA,NaN,NaN,NaN,NaN,NaN,10:00 PM,10:50 PM,NaN,NaN


In [36]:
dfv2 = main_df.drop(columns=['Time', 'Building', 'Lab_Time', 'Lab_Room','Lab_Teacher', 'Class_type', 'Lab_Building', 'Room'])

In [37]:
dfv2

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Course_Name,Code,Section,Days,Teacher,Lab_Days,Start Time,End Time,Lab Start,Lab End
0,CPCS 202 - برمجه 1,70959,AS,TRU,TBA,R,8:00 AM,8:50 AM,12:00 PM,1:20 PM
1,CPCS 202 - برمجه 1,64178,BAR,MW,TBA,R,8:00 AM,9:15 AM,9:30 AM,10:50 AM
2,CPCS 202 - برمجه 1,56801,CS1,TRU,TBA,R,1:00 PM,1:50 PM,5:30 PM,6:50 PM
3,CPCS 202 - برمجه 1,56275,CS2,MW,TBA,M,2:30 PM,3:50 PM,5:30 PM,6:50 PM
4,CPCS 202 - برمجه 1,56276,CS3,MW,TBA,U,4:00 PM,5:20 PM,11:00 AM,12:20 PM
...,...,...,...,...,...,...,...,...,...,...
75,CPIS 499 - مشروع تخرج 2,56605,S56,TRU,TBA,NaN,7:00 AM,7:50 AM,NaN,NaN
76,CPIS 499 - مشروع تخرج 2,56606,S57,TRU,TBA,NaN,8:00 AM,8:50 AM,NaN,NaN
77,CPIS 499 - مشروع تخرج 2,56607,S58,TRU,TBA,NaN,9:00 PM,9:50 PM,NaN,NaN
78,CPIS 499 - مشروع تخرج 2,56608,S59,TRU,TBA,NaN,10:00 PM,10:50 PM,NaN,NaN


In [38]:
dfv2.to_csv("courses.csv", sep=",", index=False)